<a href="https://colab.research.google.com/github/narayananv10/NLP/blob/Classwork/CountVectoriser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import nltk
from string import digits
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import lightgbm as lgb

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
df = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz",lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013"


In [0]:
twitter=pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv")

In [0]:
twitter = twitter[twitter['Sentiment']!='irrelevant']

In [0]:
def clean_text(sent):
  sent = sent.lower()
  sent = sent.translate(sent.maketrans('','',digits))
  sent = sent.translate(sent.maketrans('', '', '!"@#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'))
  return sent

In [0]:
twitter['TweetText'] = twitter['TweetText'].apply(clean_text)

In [0]:
vectoriser = CountVectorizer(lowercase=True,stop_words=set(stopwords.words('english')))
vec = vectoriser.fit_transform(twitter.TweetText)

In [0]:
features = pd.DataFrame(vec.toarray(),columns=vectoriser.get_feature_names())

In [0]:
X_train, X_test, y_train, y_test = tts(vec,twitter.Sentiment,test_size = 0.3,random_state=54)

In [0]:
train = pd.DataFrame(X_train.toarray(),columns=vectoriser.get_feature_names())
test = pd.DataFrame(X_test.toarray(),columns=vectoriser.get_feature_names())

In [0]:
rand_forest = RandomForestClassifier()
rand_forest.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred = rand_forest.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

    negative       0.81      0.37      0.51       162
     neutral       0.75      0.97      0.84       687
    positive       0.74      0.26      0.38       179

    accuracy                           0.75      1028
   macro avg       0.77      0.53      0.58      1028
weighted avg       0.76      0.75      0.71      1028



In [0]:
params = {
    "objective" : "multiclass",
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':3,
    'learning_rate':0.01,
    'max_depth': 7,
    'num_leaves': 127,
    'feature_fraction': 0.4,
    'bagging_freq': 10,
    'num_iterations':1000 ,
    'max_bin' : 32}

In [0]:
le = LabelEncoder()  
le_train = lgb.Dataset(train, le.fit_transform(y_train))

In [0]:
lgb = lgb.train(params, le_train, 100)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [0]:
pred = lgb.predict(test)
pred_test = [np.argmax(line) for line in pred]
print(classification_report(le.inverse_transform(pred_test),y_test))

              precision    recall  f1-score   support

    negative       0.22      0.47      0.30        76
     neutral       0.94      0.71      0.81       913
    positive       0.13      0.62      0.22        39

    accuracy                           0.69      1028
   macro avg       0.43      0.60      0.44      1028
weighted avg       0.86      0.69      0.75      1028



In [0]:
predtrain = lgb.predict(train)
pred_train = [np.argmax(line) for line in predtrain]
print(classification_report(le.inverse_transform(pred_train),y_train))

              precision    recall  f1-score   support

    negative       0.30      0.69      0.42       177
     neutral       0.96      0.75      0.84      2111
    positive       0.24      0.74      0.36       108

    accuracy                           0.74      2396
   macro avg       0.50      0.73      0.54      2396
weighted avg       0.88      0.74      0.79      2396

